In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install category_encoders
# !pip install dacon_submit_api-0.0.4-py3-none-any.whl

In [3]:
## import
import pandas as pd
import numpy as np
import copy

# vis
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
%matplotlib inline
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치. 코랩에선 주석하셈
import seaborn as sns

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.ensemble import StackingClassifier, StackingRegressor
from sklearn.base import ClassifierMixin
# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
# import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# 시드값 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2023)

# 데이터프레임 컬럼 전체 보기
pd.set_option('display.max_columns', None)

In [4]:
path = "./drive/MyDrive/Colab Notebooks/data/seoul_temp/"
path = '../data/'

train = pd.read_csv(path + "train.csv")
submit = pd.read_csv(path + "sample_submission.csv")

train['일시'] = pd.to_datetime(train['일시'])

In [5]:
for col in train.drop(columns='일시').columns:
  train[col] = train[col].interpolate()

### 컬럼 설명
- 일사합 : 년간? 일사량(태양의 복사 에너지가 땅에 닿는 양)의 합계
* 일조합 : 년간? 일사시간(구름이나 안개 따위에 가려지지 아니하고 햇볕이 실제로 내리쬐는 시간)의 합계
* 일조율 : rate of sunshine 일조시간과 가조시간과의 비로, 1에 가까울수록 구름이 적은 경우가 된다.

In [6]:
renames = {'일시':'time', '최고기온':'max_temp', '최저기온':'min_temp',
           '일교차':'day_diff', '강수량':'rain', '평균습도':'mean_humidity',
           '평균풍속':'mean_wind', '일사합':'total_sunshine', '일조합':'only_sunshine',
           '일조율':'sunshine_ratio',
       '평균기온':'target'}

train.rename(columns=renames, inplace=True)

여기서부터 lgbm전까지는 feature_TG, target_TG로 LSTM모델을 사용하여 학습 및 예측
<br/>
참고 : https://github.com/jungsungmoon/nongsan#eda

In [7]:
print(torch.cuda.is_available())
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(torch.cuda.get_device_name())
print(torch.cuda.device_count())

True
NVIDIA GeForce RTX 3050 Ti Laptop GPU
1


### LSTF과제에 대한 3가지 모델 : https://today-1.tistory.com/60

#### Basic LSTM Model
- https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_10_2_lstm.ipynb



In [8]:
X_train, y_train = train.drop(columns=['time','target']), train['target']

# Train-Validation split
X_tr, X_val = X_train[:-4400],X_train[-4400:]
y_tr, y_val = y_train[:-4400],y_train[-4400:]

# to tensor
X_tr_tensor = torch.tensor(X_tr.values,dtype=torch.float32).unsqueeze(1)
y_tr_tensor = torch.tensor(np.array(y_tr).reshape(-1,1),dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values,dtype=torch.float32).unsqueeze(1)
y_val_tensor = torch.tensor(np.array(y_val).reshape(-1,1),dtype=torch.float32)

# to DataLoader
train_dataset = TensorDataset(X_tr_tensor, y_tr_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# to DataLoader
BATCHSIZE = 365
train_loader = DataLoader(train_dataset, batch_size=BATCHSIZE , shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCHSIZE , shuffle=False)

display(X_tr_tensor.size(), y_tr_tensor.size()
        ,X_val_tensor.size(), y_val_tensor.size()
       )

torch.Size([18611, 1, 9])

torch.Size([18611, 1])

torch.Size([4400, 1, 9])

torch.Size([4400, 1])

### Basic LSTM forecasting

In [28]:
# define LSTM Model
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers,output_dim):
        '''
        input_dim = 피처의 개수
        hidden_dim = hidden_state의 컬럼 개수 (아무값이나)
        num_layers = lstm net의 개수 (보통 1로 지정)
        output_dim = 가격만 예측하니까 1로 지정
        '''
        super(LSTM,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers


        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                           batch_first=True)
        self.fc0 = nn.Linear(hidden_dim,64)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, output_dim)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # lstm 셀 및 가중치 초기화
        h_0 = Variable(torch.randn(self.num_layers,x.size(0),self.hidden_dim))
        c_0 = Variable(torch.randn(self.num_layers,x.size(0),self.hidden_dim))

        # out,(hn,cn) = self.lstm(x, (h_0,c_0))
        out,_ = self.lstm(x)
        out = out.view(-1,self.hidden_dim) # size = (x.size(0),hidden_dim)
        # or out=out[:,-1,:]
        out = self.fc0(out)
        out = self.fc1(out)
#         out = self.dropout(out)
        out = self.fc2(out)
        return out

class MAELoss(nn.Module):
    def __init__(self):
        super(MAELoss,self).__init__()
        
    def forward(self,y_pred,y_true):
        print(y_true, y_pred)
        print(torch.mean(torch.abs(y_pred - y_true)))
        return torch.mean(torch.abs(y_pred - y_true))
    
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False


In [29]:
# Model instance , criterion, optimizer

input_dim = X_tr_tensor.size(2) # 피처 개수
hidden_dim = 128
output_dim = 1 # 예측할 일 수
num_layers = 1

model = LSTM(input_dim, hidden_dim, num_layers, output_dim)

criterion = MAELoss() # MAE
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', 
                              factor=0.5, patience=3, verbose=True)


In [30]:
# Train the model
best_loss = float('inf')
early_stopping_counter = 0
epoch_counter = 0 # epoch 횟수 카운팅용
EPOCHS = 1000 # 훈련 횟수
patience = 70 # early_stopping 기준
train_losses_plot = []
val_losses_plot = []

es = EarlyStopping(patience=patience)
done = False

while not done and epoch_counter < EPOCHS:
    epoch_counter += 1
    # Training
    model.train() # 모델을 '학습 모드'로 설정
    train_losses = []
    for batch_x, batch_y in train_loader: # 미니 배치 학습
        # GPU
#         batch_x,batch_y = batch_x.to(device),batch_y.to(device)

        optimizer.zero_grad() # optimizer의 가중치를 초기화
        outputs = model(batch_x) # 순전파 진행
        outputs = outputs.squeeze(0)
        # output의 size는 (1, BATCHSIZE, output_dim)
        # >> squeeze(0) 해야 함 >> (BATCHSIZE, output_dim)
        loss = criterion(outputs, batch_y) # 평가(손실 계산)
        loss.backward() # 역전파 진행
        train_losses.append(loss.item())
        optimizer.step() # 모델의 가중치 업데이트
        
    train_loss = np.mean(train_losses)
    train_losses_plot.append(train_loss)
    
    # Validation
    model.eval() # 모델을 '검정 모드'로 설정
    val_losses = []
    val_loss = 0
    with torch.no_grad():
        for batch_x,batch_y in val_loader:
            # GPU
#             batch_x,batch_y = batch_x.to(device),batch_y.to(device)
            
            outputs = model(batch_x)
            outputs = outputs.squeeze(0)
            loss = criterion(outputs,batch_y)
            val_losses.append(loss.item())
    
    val_loss = np.mean(val_losses)
    val_losses_plot.append(val_loss)
    scheduler.step(val_loss) # scheduler 업데이트
    
    if es(model, val_loss):
        done = True
    
    if (epoch_counter % 10 == 0) : # 10번마다 출력
        print(f'EPOCH : {epoch_counter} , train_loss : {train_loss}, val_loss : {val_loss}, {es.status}')    

tensor([[ -1.6000],
        [ -1.9000],
        [  4.0000],
        [  7.5000],
        [ -4.6000],
        [ -5.2000],
        [  1.3000],
        [  1.9000],
        [  2.3000],
        [  2.2000],
        [ -2.3000],
        [ -3.8000],
        [ -2.8000],
        [ -1.5000],
        [ -4.8000],
        [ -5.0000],
        [ -5.3000],
        [ -2.5000],
        [  2.1000],
        [ -0.5000],
        [ -6.1000],
        [ -9.5000],
        [-13.1000],
        [-12.5000],
        [ -7.3000],
        [ -7.2000],
        [-10.5000],
        [-11.0000],
        [ -7.5000],
        [ -0.2000],
        [  2.6000],
        [  0.8000],
        [  1.2000],
        [  1.2000],
        [ -1.2000],
        [  1.3000],
        [  6.7000],
        [  9.2000],
        [  9.1000],
        [  2.9000],
        [ -3.1000],
        [ -3.3000],
        [ -4.1000],
        [  0.6000],
        [ -0.8000],
        [ -1.1000],
        [ -0.4000],
        [  1.8000],
        [ -1.3000],
        [  3.4000],


        [ -4.6000]]) tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
    

        [  2.3000]]) tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
    

tensor([[ 3.4000],
        [ 3.2000],
        [ 3.2000],
        [ 1.0000],
        [-5.7000],
        [-8.1000],
        [-5.6000],
        [-0.4000],
        [ 1.8000],
        [ 1.0000],
        [-1.9000],
        [-2.4000],
        [ 0.2000],
        [ 3.9000],
        [ 2.3000],
        [ 0.9000],
        [ 0.4000],
        [ 0.4000],
        [ 0.2000],
        [ 2.2000],
        [ 1.8000],
        [ 0.0000],
        [-1.3000],
        [-0.8000],
        [-4.4000],
        [-5.7000],
        [-5.0000],
        [-3.4000],
        [-0.1000],
        [-0.4000],
        [-2.8000],
        [-0.8000],
        [ 1.9000],
        [ 0.3000],
        [ 2.0000],
        [ 2.3000],
        [ 2.1000],
        [ 0.4000],
        [-1.1000],
        [-0.9000],
        [-0.6000],
        [ 0.7000],
        [-1.7000],
        [-1.9000],
        [-2.1000],
        [-3.4000],
        [-3.7000],
        [-1.8000],
        [ 1.2000],
        [-1.9000],
        [-0.2000],
        [-1.0000],
        [ 1.

tensor([[ -1.2000],
        [  3.3000],
        [  4.5000],
        [  2.5000],
        [  0.9000],
        [ -3.8000],
        [ -3.0000],
        [  2.3000],
        [  4.2000],
        [  0.3000],
        [  2.7000],
        [  3.7000],
        [  3.1000],
        [  3.9000],
        [  1.8000],
        [  2.9000],
        [  4.5000],
        [  1.7000],
        [  0.3000],
        [  1.8000],
        [ -0.3000],
        [ -0.6000],
        [ -0.7000],
        [ -6.7000],
        [ -5.1000],
        [ -1.3000],
        [  1.4000],
        [  1.2000],
        [  0.3000],
        [ -1.0000],
        [ -4.4000],
        [-14.3000],
        [-13.0000],
        [ -8.7000],
        [ -7.3000],
        [ -7.0000],
        [ -5.9000],
        [ -4.1000],
        [ -0.4000],
        [  2.3000],
        [  1.7000],
        [  1.9000],
        [  3.1000],
        [ -1.7000],
        [ -4.2000],
        [ -1.4000],
        [ -4.9000],
        [ -4.0000],
        [ -3.6000],
        [ -0.2000],


...

tensor([[  6.5000],
        [  6.4000],
        [  5.8000],
        [  2.3000],
        [ -2.6000],
        [ -6.3000],
        [ -4.5000],
        [ -0.9000],
        [ -2.4000],
        [ -2.0000],
        [ -1.4000],
        [ -4.7000],
        [ -4.3000],
        [ -3.6000],
        [-10.2000],
        [-12.8000],
        [ -9.2000],
        [ -5.6000],
        [ -3.3000],
        [  0.5000],
        [  0.7000],
        [  0.6000],
        [  3.9000],
        [  5.5000],
        [ -2.2000],
        [ -3.9000],
        [ -0.1000],
        [  2.8000],
        [  1.9000],
        [  0.9000],
        [ -0.7000],
        [ -2.1000],
        [ -2.1000],
        [ -1.3000],
        [ -1.6000],
        [  0.3000],
        [  2.7000],
        [ -1.7000],
        [ -8.1000],
        [-11.7000],
        [ -7.2000],
        [ -2.5000],
        [ -1.3000],
        [ -0.6000],
        [ -0.6000],
        [  0.6000],
        [ -0.3000],
        [  0.5000],
        [  4.3000],
        [  4.5000],


tensor([[-4.9000],
        [-4.4000],
        [-3.6000],
        [-0.3000],
        [ 2.2000],
        [-3.5000],
        [-4.0000],
        [-3.1000],
        [-2.1000],
        [-1.1000],
        [-1.6000],
        [-0.5000],
        [-0.5000],
        [-6.8000],
        [-6.5000],
        [-2.9000],
        [ 2.2000],
        [ 3.8000],
        [ 4.0000],
        [ 1.5000],
        [ 2.3000],
        [-0.1000],
        [-0.8000],
        [ 0.8000],
        [ 3.0000],
        [ 0.5000],
        [-7.8000],
        [-4.7000],
        [-1.7000],
        [-2.4000],
        [-6.2000],
        [-4.3000],
        [-2.9000],
        [ 0.1000],
        [-0.1000],
        [-2.2000],
        [-2.6000],
        [-1.5000],
        [-5.5000],
        [-3.9000],
        [-0.8000],
        [ 3.0000],
        [ 4.2000],
        [-2.1000],
        [ 0.2000],
        [ 1.5000],
        [ 2.2000],
        [ 4.0000],
        [ 1.3000],
        [ 4.9000],
        [ 4.9000],
        [-3.5000],
        [-7.

tensor([[  0.5000],
        [ -4.5000],
        [ -3.4000],
        [ -2.7000],
        [ -4.1000],
        [ -2.5000],
        [  0.6000],
        [ -1.8000],
        [  0.0000],
        [  1.5000],
        [  0.6000],
        [  4.5000],
        [  6.3000],
        [  3.6000],
        [  0.3000],
        [  1.9000],
        [  3.2000],
        [  1.8000],
        [ -2.6000],
        [ -5.5000],
        [ -3.4000],
        [ -2.0000],
        [  0.0000],
        [ -0.5000],
        [  0.2000],
        [ -0.8000],
        [ -0.3000],
        [  2.8000],
        [  4.6000],
        [  2.6000],
        [  0.0000],
        [ -4.3000],
        [ -5.4000],
        [ -6.0000],
        [ -5.9000],
        [ -4.8000],
        [ -1.7000],
        [ -0.2000],
        [  0.5000],
        [  0.8000],
        [  0.3000],
        [ -0.2000],
        [ -8.3000],
        [-12.4000],
        [-11.1000],
        [ -7.6000],
        [ -5.8000],
        [ -4.5000],
        [ -5.4000],
        [ -4.3000],


tensor([[ -4.2000],
        [ -5.2000],
        [ -3.4000],
        [ -0.8000],
        [  1.0000],
        [  3.0000],
        [  3.0000],
        [  1.3000],
        [ -1.1000],
        [ -1.5000],
        [ -0.5000],
        [  1.4000],
        [  1.7000],
        [  0.8000],
        [  0.2000],
        [ -0.8000],
        [ -1.6000],
        [  0.1000],
        [ -2.0000],
        [ -6.0000],
        [ -3.6000],
        [ -4.6000],
        [ -2.2000],
        [ -0.9000],
        [ -0.2000],
        [ -1.5000],
        [-11.2000],
        [-11.2000],
        [ -9.1000],
        [ -7.2000],
        [ -6.8000],
        [ -8.5000],
        [ -4.8000],
        [ -1.2000],
        [ -2.2000],
        [ -4.2000],
        [ -2.7000],
        [  0.3000],
        [  0.7000],
        [  2.0000],
        [ -0.8000],
        [ -1.6000],
        [ -4.3000],
        [ -6.5000],
        [ -5.5000],
        [ -8.6000],
        [ -6.1000],
        [ -2.7000],
        [  1.9000],
        [  1.8000],


KeyboardInterrupt: 

In [ ]:
# final evaluation
model.eval()
with torch.no_grad():
    pred = model(X_val_tensor)
score = torch.mean(torch.abs(pred.squeeze(0)-y_val_tensor))
print("final score : {:.5f}".format(score))